## Setup

Step 1: Download the two models from this link and store them in `./checkpoints`.

Step 2: Use this notebook to load the models.

Step 3: If needed, convert the models to a HuggingFace version. (this is TransformerLens and might have different variable names, but the model is functionally identical to gpt2-small.)

Step 4: Interp.

In [2]:
from transformer_lens import HookedTransformer
import torch
from plotly import graph_objects as go
import plotly.express as px
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
path = './checkpoints/'

In [7]:
# clustered model

clustered_model = HookedTransformer.from_pretrained("gpt2-small")
clustered_model.to(device)
clustered_model.load_state_dict(torch.load(path + 'wiki_fully_modular_mlp_in_out.pt', map_location=device))

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


/tmp/ipykernel_1124054/1954707113.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clustered_model.load_state_dict(torch.load(path + 'wiki_fully_modular_mlp_in_out.pt', m

<All keys matched successfully>

In [10]:
# unclustered model

unclustered_model = HookedTransformer.from_pretrained("gpt2-small")
unclustered_model.to(device)
unclustered_model.load_state_dict(torch.load(path + 'wiki_non_modular_mlp_in_out.pt', map_location=device))

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


/tmp/ipykernel_1124054/2121264898.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unclustered_model.load_state_dict(torch.load(path + 'wiki_non_modular_mlp_in_out.pt', m

<All keys matched successfully>

In [12]:
# sanity check to see if the models are loaded correctly

input = 'The quick brown fox'

clustered_output = clustered_model.generate(input, max_new_tokens=20)
unclustered_output = unclustered_model.generate(input, max_new_tokens=20)

print('Clustered model output: ', clustered_output)

print('Unclustered model output: ', unclustered_output)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Clustered model output:  The quick brown foxes that were commonly passed in Virginia changed their name to the American coyote Herbert in 1840 , so
Unclustered model output:  The quick brown fox with <unk> blue @-@ white teeth and a wicked <unk> mouth was quietly preserved


In [ ]:
# enjoy interp!